## Set up

In [1]:
import pandas as pd
import numpy as np
import plotly_express as px
import dask.dataframe as dd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn import preprocessing, metrics
from ipywidgets import widgets, interactive
import gc
import joblib
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta 
from typing import Union
from tqdm.notebook import tqdm_notebook as tqdm
from itertools import cycle
import datetime as dt
from torch.autograd import Variable
import random 
import os
from matplotlib.pyplot import figure
from fastprogress import master_bar, progress_bar
import torch
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import time 
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import torch 

%matplotlib inline

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
sales = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
sales.name = 'sales'
calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
calendar.name = 'calendar'
prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
prices.name = 'prices'

## Downcasting

In [3]:
sales_bd = np.round(sales.memory_usage().sum()/(1024*1024),1)
calendar_bd = np.round(calendar.memory_usage().sum()/(1024*1024),1)
prices_bd = np.round(prices.memory_usage().sum()/(1024*1024),1)

In [4]:
import numpy as np
#Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

sales = downcast(sales)

In [5]:
sales_ad = np.round(sales.memory_usage().sum()/(1024*1024),1)
calendar_ad = np.round(calendar.memory_usage().sum()/(1024*1024),1)
prices_ad = np.round(prices.memory_usage().sum()/(1024*1024),1)

In [ ]:
dic = {'DataFrame':['sales','calendar','prices'],
       'Before downcasting':[sales_bd,calendar_bd,prices_bd],
       'After downcasting':[sales_ad,calendar_ad,prices_ad]}

memory = pd.DataFrame(dic)
memory = pd.melt(memory, id_vars='DataFrame', var_name='Status', value_name='Memory (MB)')
memory.sort_values('Memory (MB)',inplace=True)
fig = px.bar(memory, x='DataFrame', y='Memory (MB)', color='Status', barmode='group', text='Memory (MB)')
fig.update_traces(texttemplate='%{text} MB', textposition='outside')
fig.update_layout(template='seaborn', title='Effect of Downcasting')
fig.show()

In [6]:
#Create date index
date_index = calendar['date']
dates = date_index[0:1913]
dates_list = [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in dates]

In [7]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [8]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1,0,1,4,0,0,5,0,0,0,0,0,0,2,1,2,1,0,0,0,1,1,1,0,0,1,1,1,1

In [9]:
df = sales.T
df.head(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,30240,30241,30242,30243,30244,30245,30246,30247,30248,30249,30250,30251,30252,30253,30254,30255,30256,30257,30258,30259,30260,30261,30262,30263,30264,30265,30266,30267,30268,30269,30270,30271,30272,30273,30274,30275,30276,30277,30278,30279,30280,30281,30282,30283,30284,30285,30286,30287,30288,30289,30290,30291,30292,30293,30294,30295,30296,30297,30298,30299,30300,30301,30302,30303,30304,30305,30306,30307,30308,30309,30310,30311,30312,30313,30314,30315,30316,30317,30318,30319,30320,30321,30322,30323,30324,30325,30326,30327,30328,30329,30330,30331,30332,30333,30334,30335,30336,30337,30338,30339,30340,30341,30342,30343,30344,30345,30346,30347,30348,30349,30350,30351,30352,30353,30354,30355,30356,30357,30358,30359,30360,30361,30362,30363,30364,30365,30366,30367,30368,30369,30370,30371,30372,30373,30374,30375,30376,30377,30378,30379,30380,30381,30382,30383,30384,30385,30386,30387,30388,30389,30390,30391,30392,30393,30394,30395,30396,30397,30398,30399,30400,30401,30402,30403,30404,30405,30406,30407,30408,30409,30410,30411,30412,30413,30414,30415,30416,30417,30418,30419,30420,30421,30422,30423,30424,30425,30426,30427,30428,30429,30430,30431,30432,30433,30434,30435,30436,30437,30438,30439,30440,30441,30442,30443,30444,30445,30446,30447,30448,30449,30450,30451,30452,30453,30454,30455,30456,30457,30458,30459,30460,30461,30462,30463,30464,30465,30466,30467,30468,30469,30470,30471,30472,30473,30474,30475,30476,30477,30478,30479,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_011_CA_1_evaluation,HOBBIES_1_012_CA_1_evaluation,HOBBIES_1_013_CA_1_evaluation,HOBBIES_1_014_CA_1_evaluation,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_016_CA_1_evaluation,HOBBIES_1_017_CA_1_evaluation,HOBBIES_1_018_CA_1_evaluation,HOBBIES_1_019_CA_1_evaluation,HOBBIES_1_020_CA_1_evaluation,HOBBIES_1_021_CA_1_evaluation,HOBBIES_1_022_CA_1_evaluation,HOBBIES_1_023_CA_1_evaluation,HOBBIES_1_024_CA_1_evaluation,HOBBIES_1_025_CA_1_evaluation,HOBBIES_1_026_CA_1_evaluation,HOBBIES_1_027_CA_1_evaluation,HOBBIES_1_028_CA_1_evaluation,HOBBIES_1_029_CA_1_evaluation,HOBBIES_1_030_CA_1_evaluation,HOBBIES_1_031_CA_1_evaluation,HOBBIES_1_032_CA_1_evaluation,HOBBIES_1_033_CA_1_evaluation,HOBBIES_1_034_CA_1_evaluation,HOBBIES_1_035_CA_1_evaluation,HOBBIES_1_036_CA_1_evaluation,HOBBIES_1_037_CA_1_evaluation,HOBBIES_1_038_CA_1_evaluation,HOBBIES_1_039_CA_1_evaluation,HOBBIES_1_040_CA_1_evaluation,HOBBIES_1_041_CA_1_evaluation,HOBBIES_1_042_CA_1_evaluation,HOBBIES_1_043_CA_1_evaluation,HOBBIES_1_044_CA_1_evaluation,HOBBIES_1_045_CA_1_evaluation,HOBBIES_1_046_CA_1_evaluation,HOBBIES_1_047_CA_1_evaluation,HOBBIES_1_048_CA_1_evaluation,HOBBIES_1_049_CA_1_evaluation,HOBBIES_1_050_CA_1_evaluation,HOBBIES_1_051_CA_1_evaluation,HOBBIES_1_052_CA_1_evaluation,HOBBIES_1_053_CA_1_evaluation,HOBBIES_1_05

## Data Enineering

In [10]:
timesteps = 7
startDay = 350
# drop the categorical rows
df = df[6 + startDay:]
df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,30240,30241,30242,30243,30244,30245,30246,30247,30248,30249,30250,30251,30252,30253,30254,30255,30256,30257,30258,30259,30260,30261,30262,30263,30264,30265,30266,30267,30268,30269,30270,30271,30272,30273,30274,30275,30276,30277,30278,30279,30280,30281,30282,30283,30284,30285,30286,30287,30288,30289,30290,30291,30292,30293,30294,30295,30296,30297,30298,30299,30300,30301,30302,30303,30304,30305,30306,30307,30308,30309,30310,30311,30312,30313,30314,30315,30316,30317,30318,30319,30320,30321,30322,30323,30324,30325,30326,30327,30328,30329,30330,30331,30332,30333,30334,30335,30336,30337,30338,30339,30340,30341,30342,30343,30344,30345,30346,30347,30348,30349,30350,30351,30352,30353,30354,30355,30356,30357,30358,30359,30360,30361,30362,30363,30364,30365,30366,30367,30368,30369,30370,30371,30372,30373,30374,30375,30376,30377,30378,30379,30380,30381,30382,30383,30384,30385,30386,30387,30388,30389,30390,30391,30392,30393,30394,30395,30396,30397,30398,30399,30400,30401,30402,30403,30404,30405,30406,30407,30408,30409,30410,30411,30412,30413,30414,30415,30416,30417,30418,30419,30420,30421,30422,30423,30424,30425,30426,30427,30428,30429,30430,30431,30432,30433,30434,30435,30436,30437,30438,30439,30440,30441,30442,30443,30444,30445,30446,30447,30448,30449,30450,30451,30452,30453,30454,30455,30456,30457,30458,30459,30460,30461,30462,30463,30464,30465,30466,30467,30468,30469,30470,30471,30472,30473,30474,30475,30476,30477,30478,30479,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,0,1,0,4,17,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,16,0,0,0,0,0,2,0,0,0,0,0,0,0,0,4,6,0,0,0,0,0,0,3,0,0,0,0,0,0,0,3,0,0,4,1,0,0,1,0,0,2,0,0,1,1,14,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,8,0,0,0,0,0,0,4,7,0,0,0,0,1,0,1,2,0,0,4,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,27,0,0,1,0,0,0,0,0,9,2,0,0,0,0,0,0,0,0,14,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,3,0,1,4,0,0,0,4,0,0,0,13,0,0,0,0,0,8,0,3,1,...,0,0,2,0,0,103,1,0,0,6,0,77,52,0,0,3,0,5,0,0,0,4,0,0,0,0,0,0,0,0,3,0,0,5,0,6,16,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,8,0,0,6,2,0,11,12,0,0,0,0,0,0,0,0,0,0,2,0,3,0,0,2,2,0,0,6,5,3,0,2,0,0,0,9,16,0,4,0,21,5,1,0,0,0,3,4,6,4,2,2,0,33,0,0,0,6,0,2,0,0,0,0,5,0,45,5,0,7,0,3,0,8,0,6,2,0,0,3,0,0,85,1,0,18,3,2,0,10,0,0,4,0,51,0,5,0,2,1,8,0,6,0,8,0,8,0,8,0,6,0,0,16,0,15,0,3,8,0,0,1,3,0,5,4,16,0,0,0,0,0,2,25,3,2,4,0,0,0,0,2,0,0,0,0,0,3,0,0,2,18,7,17,0,3,0,0,0,0,0,14,0,1,0,0,1,0,1,60,4,2,0,10,2,0,9,2,9,0,0,7,0,6,0,0,0,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,0,0,0,2,0,0,3,0,0,0,0,1,2,0,0,0,0,0,0,0,0,9,0,0,0,3,0,1,0,0,1,5,6,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,1,0,0,0,0,0,0,3,0,1,0,2,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,1,7,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,1,0,3,0,0,0,0,0,0,1,12,0,2,2,0,2,0,2,0,0,0,1,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,2,0,0,1,0,0,0,0,0,12,1,0,0,0,0,0,0,0,0,10,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,3,2,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,2,2,0,0,0,0,0,0,0,5,1,0,0,0,0,13,0,4,0,...,0,0,3,0,0,77,1,0,0,5,3,76,53,0,0,7,0,3,0,1,0,10,0

In [11]:
daysBeforeEvent = pd.DataFrame(np.zeros((1969,1)))

In [12]:
daysBeforeEvent.value_counts()

0.0    1969
Name: count, dtype: int64

In [13]:
import datetime as dt
for i,j in calendar.iterrows():
    if((pd.isnull(calendar["event_name_1"][i]) and pd.isnull(calendar["event_name_2"][i])) == False):
           daysBeforeEvent[0][i-1] = 1 
#if first day was an event this row will cause an exception because "x-1".
#Since it is not i did not consider for now

In [14]:
daysBeforeEvent.value_counts()

0.0    1807
1.0     162
Name: count, dtype: int64

In [15]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,30240,30241,30242,30243,30244,30245,30246,30247,30248,30249,30250,30251,30252,30253,30254,30255,30256,30257,30258,30259,30260,30261,30262,30263,30264,30265,30266,30267,30268,30269,30270,30271,30272,30273,30274,30275,30276,30277,30278,30279,30280,30281,30282,30283,30284,30285,30286,30287,30288,30289,30290,30291,30292,30293,30294,30295,30296,30297,30298,30299,30300,30301,30302,30303,30304,30305,30306,30307,30308,30309,30310,30311,30312,30313,30314,30315,30316,30317,30318,30319,30320,30321,30322,30323,30324,30325,30326,30327,30328,30329,30330,30331,30332,30333,30334,30335,30336,30337,30338,30339,30340,30341,30342,30343,30344,30345,30346,30347,30348,30349,30350,30351,30352,30353,30354,30355,30356,30357,30358,30359,30360,30361,30362,30363,30364,30365,30366,30367,30368,30369,30370,30371,30372,30373,30374,30375,30376,30377,30378,30379,30380,30381,30382,30383,30384,30385,30386,30387,30388,30389,30390,30391,30392,30393,30394,30395,30396,30397,30398,30399,30400,30401,30402,30403,30404,30405,30406,30407,30408,30409,30410,30411,30412,30413,30414,30415,30416,30417,30418,30419,30420,30421,30422,30423,30424,30425,30426,30427,30428,30429,30430,30431,30432,30433,30434,30435,30436,30437,30438,30439,30440,30441,30442,30443,30444,30445,30446,30447,30448,30449,30450,30451,30452,30453,30454,30455,30456,30457,30458,30459,30460,30461,30462,30463,30464,30465,30466,30467,30468,30469,30470,30471,30472,30473,30474,30475,30476,30477,30478,30479,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,0,1,0,4,17,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,16,0,0,0,0,0,2,0,0,0,0,0,0,0,0,4,6,0,0,0,0,0,0,3,0,0,0,0,0,0,0,3,0,0,4,1,0,0,1,0,0,2,0,0,1,1,14,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,8,0,0,0,0,0,0,4,7,0,0,0,0,1,0,1,2,0,0,4,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,27,0,0,1,0,0,0,0,0,9,2,0,0,0,0,0,0,0,0,14,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,3,0,1,4,0,0,0,4,0,0,0,13,0,0,0,0,0,8,0,3,1,...,0,0,2,0,0,103,1,0,0,6,0,77,52,0,0,3,0,5,0,0,0,4,0,0,0,0,0,0,0,0,3,0,0,5,0,6,16,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,8,0,0,6,2,0,11,12,0,0,0,0,0,0,0,0,0,0,2,0,3,0,0,2,2,0,0,6,5,3,0,2,0,0,0,9,16,0,4,0,21,5,1,0,0,0,3,4,6,4,2,2,0,33,0,0,0,6,0,2,0,0,0,0,5,0,45,5,0,7,0,3,0,8,0,6,2,0,0,3,0,0,85,1,0,18,3,2,0,10,0,0,4,0,51,0,5,0,2,1,8,0,6,0,8,0,8,0,8,0,6,0,0,16,0,15,0,3,8,0,0,1,3,0,5,4,16,0,0,0,0,0,2,25,3,2,4,0,0,0,0,2,0,0,0,0,0,3,0,0,2,18,7,17,0,3,0,0,0,0,0,14,0,1,0,0,1,0,1,60,4,2,0,10,2,0,9,2,9,0,0,7,0,6,0,0,0,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,0,0,0,2,0,0,3,0,0,0,0,1,2,0,0,0,0,0,0,0,0,9,0,0,0,3,0,1,0,0,1,5,6,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,1,0,0,0,0,0,0,3,0,1,0,2,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,1,7,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,1,0,3,0,0,0,0,0,0,1,12,0,2,2,0,2,0,2,0,0,0,1,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,2,0,0,1,0,0,0,0,0,12,1,0,0,0,0,0,0,0,0,10,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,3,2,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,2,2,0,0,0,0,0,0,0,5,1,0,0,0,0,13,0,4,0,...,0,0,3,0,0,77,1,0,0,5,3,76,53,0,0,7,0,3,0,1,0,10,0

In [16]:
daysBeforeEventTest = daysBeforeEvent[1941:1969]
#"daysBeforeEvent" will be used for training as a feature.
daysBeforeEvent = daysBeforeEvent[startDay:1941]

In [17]:
import datetime
daysBeforeEvent.columns = ["oneDayBeforeEvent"]
daysBeforeEvent.index = df.index

In [18]:
df = pd.concat([df, daysBeforeEvent], axis = 1)

df.columns

Index([                  0,                   1,                   2,
                         3,                   4,                   5,
                         6,                   7,                   8,
                         9,
       ...
                     30481,               30482,               30483,
                     30484,               30485,               30486,
                     30487,               30488,               30489,
       'oneDayBeforeEvent'],
      dtype='object', length=30491)

In [19]:
df = df.astype(float)

In [20]:
df = df.apply(pd.to_numeric, errors='coerce')

In [21]:
df.columns = df.columns.astype(str)

In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))
df_scaled = scaler.fit_transform(df)

In [23]:
X_train = []
y_train = []
for i in range(timesteps, 1941 - startDay):
    X_train.append(df_scaled[i-timesteps:i])
    y_train.append(df_scaled[i][0:30490]) 

In [24]:
del df_scaled

In [25]:
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(1584, 7, 30491)
(1584, 30490)


## Modeling

In [26]:
# reshape
input_shape = (X_train.shape[1], X_train.shape[2])

In [27]:
input_shape

(7, 30491)

In [29]:
input_unit_size = 1584*7

# Importing the Keras libraries and packages
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


# Initialising the model setting
model = Sequential()

# Adding the 1st LSTM layer and some Dropout regularisation
model.add(LSTM(units = 550, return_sequences = True, input_dim = input_unit_size,activation='relu',
                   input_shape = input_shape))
model.add(Dropout(0.25))
# Adding a 2nd LSTM layer and some Dropout regularisation
model.add(LSTM(units = 400, return_sequences = True, activation='relu'))
model.add(Dropout(0.25))

# Adding a 3rd LSTM layer and some Dropout regularisation
model.add(LSTM(units = 400, return_sequences = False, activation='relu'))

# Adding the output layer
model.add(Dense(units = 30490))
    
# Compiling
model.compile(optimizer = 'adam', loss = 'mse', metrics=['mean_squared_error'])

In [34]:
# Fit model ans stack
model.fit(X_train, y_train, epochs = 150, batch_size = 128, verbose=2)

Epoch 1/150
13/13 - 2s - loss: 0.0095 - mean_squared_error: 0.0095 - 2s/epoch - 163ms/step
Epoch 2/150
13/13 - 2s - loss: 0.0095 - mean_squared_error: 0.0095 - 2s/epoch - 137ms/step
Epoch 3/150
13/13 - 2s - loss: 0.0095 - mean_squared_error: 0.0095 - 2s/epoch - 138ms/step
Epoch 4/150
13/13 - 2s - loss: 0.0095 - mean_squared_error: 0.0095 - 2s/epoch - 138ms/step
Epoch 5/150
13/13 - 2s - loss: 0.0095 - mean_squared_error: 0.0095 - 2s/epoch - 137ms/step
Epoch 6/150
13/13 - 2s - loss: 0.0095 - mean_squared_error: 0.0095 - 2s/epoch - 139ms/step
Epoch 7/150
13/13 - 2s - loss: 0.0095 - mean_squared_error: 0.0095 - 2s/epoch - 140ms/step
Epoch 8/150
13/13 - 2s - loss: 0.0095 - mean_squared_error: 0.0095 - 2s/epoch - 138ms/step
Epoch 9/150
13/13 - 2s - loss: 0.0095 - mean_squared_error: 0.0095 - 2s/epoch - 137ms/step
Epoch 10/150
13/13 - 2s - loss: 0.0094 - mean_squared_error: 0.0094 - 2s/epoch - 136ms/step
Epoch 11/150
13/13 - 2s - loss: 0.0094 - mean_squared_error: 0.0094 - 2s/epoch - 140ms/st

In [ ]:
his_loss = model.history['loss']

In [31]:
inputs= df[-timesteps:]
inputs = scaler.transform(inputs)

In [32]:
X_test = []
X_test.append(inputs[0:14])
X_test = np.array(X_test)
predictions = []

for j in range(timesteps,timesteps + 28):
#X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    y_predict = model.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30491))
    testappend = np.column_stack((np.array(y_predict), daysBeforeEventTest[0][1941 + j - timesteps]))
    X_test = np.append(X_test, testappend).reshape(1,j + 1,30491)
    y_predict = scaler.inverse_transform(testappend)[:,0:30490]
    predictions.append(y_predict)

1/1 [==============================] - 0s 26ms/step


### Submission

In [33]:
import time

submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))

submission = submission.T
    
submission = pd.concat((submission, submission), ignore_index=True)

sample_submission = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")
    
idColumn = sample_submission[["id"]]
    
submission[["id"]] = idColumn  

cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]

colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]

submission.columns = colsdeneme

currentDateTime = time.strftime("%d%m%Y_%H%M%S")

submission.to_csv("submission.csv", index=False)